In [1]:
import numpy as np
from PIL import Image
import scipy.io
import random as rn
import matplotlib.pyplot as plt
from numpy import linalg as la
import math as m
from scipy.io import wavfile
import scipy.io
import IPython
import librosa as lb
import scipy

#### Question 2

In [2]:
#loading the piano and ocean files using librosa.load
audio1, SR1 = lb.load('./data/trs.wav',sr=None)
audio2, SR2 = lb.load('./data/trn.wav',sr=None)
audio3, SR3 = lb.load('./data/tex.wav',sr=None)
audio4, SR4 = lb.load('./data/tes.wav',sr=None)

In [3]:
print(SR1,SR2,SR3)

16000 16000 16000


In [4]:
IPython.display.Audio(np.real(audio3),rate=SR3)

In [5]:
#Transforming the orginal sound wave into an stft matrix using Hann window
def stft(audio):
    return lb.stft(audio,n_fft=1024, hop_length=512, win_length=1024, window='hann', center=True)

#Transforming an stft matrix into an original sound wave using Hann window
def stftInverse(audio):
    return lb.istft(audio, hop_length=512, center=True)

def initB(audio,K):
    B = np.random.randn(audio.shape[0], K)
    Theta = np.random.randn(K,audio.shape[1])
    bOne = np.ones((B.shape[0],B.shape[0]))
    thetaOne = np.ones((Theta.shape[0],Theta.shape[0]))
    B = B/(np.dot(bOne,B)+ 0.0000000001)
    Theta = Theta/(np.dot(thetaOne,Theta) + 0.0000000001)
    
    for i in range(300):
        
        mult1 = np.dot(B,Theta) + 0.0000000001
        div1 = np.divide(audio,mult1)
        mul1 = np.dot(div1,Theta.T)
        B = np.multiply(B,mul1)
        
        mult2 = np.dot(B,Theta) + 0.0000000001
        div2 = np.divide(audio,mult2)
        mul2 = np.dot(B.T,div2)
        Theta = np.multiply(Theta,mul2)
        
        B = B/(np.dot(bOne,B) + 0.0000000001)
        Theta = Theta/(np.dot(thetaOne,Theta) + 0.0000000001)

    return (B,Theta)

def initTheta(audio,K,B):
    Theta = np.random.randn(K,audio.shape[1])
    thetaOne = np.ones((Theta.shape[0],Theta.shape[0]))
    Theta = Theta/np.dot(thetaOne,Theta)
    
    for i in range(300):
        mult2 = np.dot(B,Theta) + 0.0000000001
        div2 = np.divide(audio,mult2)
        mul2 = np.dot(B.T,div2)
        Theta = np.multiply(Theta,mul2)
        Theta = Theta/(np.dot(thetaOne,Theta) + 0.0000000001)

    return Theta

def SNR(p1,p2):
    diff = p1 - p2
    frac = np.dot(p1.T,p1)/np.dot(diff.T,diff)
    SNR = 10 * np.log10(frac)
    return SNR

In [6]:
S,N,X,Y = stft(audio1),stft(audio2),stft(audio3),stft(audio4)

In [7]:
print(S.shape,N.shape,X.shape,Y.shape)

(513, 893) (513, 893) (513, 161) (513, 161)


In [8]:
Ks,Kn = 30,30

In [9]:
B1, Theta1 = initB(abs(S),Ks)
B1

array([[ 0.,  0.,  0., ...,  0., -0., -0.],
       [ 0.,  0., -0., ..., -0.,  0., -0.],
       [ 0., -0.,  0., ...,  0., -0., -0.],
       ...,
       [ 0.,  0.,  0., ...,  0., -0.,  0.],
       [ 0., -0.,  0., ...,  0.,  0., -0.],
       [-0., -0.,  0., ..., -0.,  0.,  0.]])

In [10]:
B2, Theta2 = initB(abs(N),Kn)
B2

array([[-0., -0.,  0., ...,  0.,  0., -0.],
       [ 0., -0., -0., ..., -0., -0., -0.],
       [-0., -0.,  0., ..., -0., -0., -0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0., -0.],
       [-0., -0.,  0., ...,  0., -0., -0.],
       [-0., -0., -0., ...,  0., -0., -0.]])

In [11]:
B3 = np.hstack((B1,B2))
B3.shape

(513, 60)

In [12]:
Theta3 = initTheta(abs(X),Ks+Kn,B3)
Theta3.shape

(60, 161)

In [16]:
probMat = np.dot(B1,Theta3[:Ks,:])
S1 = probMat/(np.dot(B3,Theta3) + 0.0000000001)
Shat = np.multiply(S1,Y)

In [17]:
S_recovered = stftInverse(Shat)
IPython.display.Audio(np.real(S_recovered),rate=SR3)

In [18]:
SNR1 = SNR(audio4,S_recovered)
SNR1

9.855949537968337